In [1]:
import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.zoo as foz

dataset = foz.load_zoo_dataset("quickstart")

# Reset the brain run indexes - only run if this is a new mongodb instance for fifty one OR
# you manually deleted a brain run
# dataset.delete_brain_run("es_idx_clip")
# dataset.delete_brain_run("es_idx_alex")
# dataset.delete_brain_run("clip_viz")
# dataset.delete_brain_run("alex_viz")


Dataset already downloaded
Loading existing dataset 'quickstart'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


ValueError: Dataset has no brain method run key 'es_idx_alex'

Loaded the dataset now let's open the view

In [ ]:
# auto=false prevents the app interface from opening in the notebook
session = fo.launch_app(dataset, auto=False)
            

Now we make the embeddings and put them into elasticsearch

In [ ]:
es_idx_clip = fob.compute_similarity(
    dataset,    
    model="clip-vit-base32-torch",
    brain_key="es_idx_clip",
    backend="elasticsearch",
    hosts=["http://localhost:9200"],           
    username= "elastic",
    password="cdBfWA3A",                                               
    metric="cosine"
)

In [ ]:
es_idx_alex = fob.compute_similarity(
    dataset,
    model="alexnet-imagenet-torch",
    brain_key="es_idx_alex",
    backend="elasticsearch",
    hosts=["http://localhost:9200"],
    username= "elastic",
    password="cdBfWA3A",
    metric="cosine"
)

Time to do a query and change the session view

In [ ]:
# query = ("a cat"    )
# view2 = dataset.sort_by_similarity(query, k=10, brain_key="es_idx_clip")
# session.view = view2    

# # Query by sample ID
# query = dataset.first().id
# view = dataset.sort_by_similarity(query, k=10, brain_key="es_idx_clip")
# session.view = view     


Finally let's look at the 2D scatter plot of the embeddings. Note, there are several ways to reduce the embedding dimensions to 2, FiftyOne uses Uniform Manifold Approximation and Projection (UMAP) by default https://docs.voxel51.com/brain.html#dimensionality-reduction-methods

In [ ]:
clip_viz = fob.compute_visualization(dataset, embeddings=es_idx_clip, brain_key='clip_viz')
clip_alex = fob.compute_visualization(dataset, embeddings=es_idx_alex, brain_key='alex_viz')


### Notes on running this demo

If you do multiple runs of the model look at this doc to explain the already exisiting run index
https://voxel51.com/blog/fiftyone-computer-vision-tips-and-tricks-july-28-2023/

To start the demo you need to 
1. start up the mongodb container in podman
2. use the elasticsearch script in ~/bin/elastic-start-local 
3. repo https://github.com/elastic/elastic-start-local 
4. `docker compose up -d`
5. `docker compose stop`
6. `docker compose up --wait`  (restarts the service
7. password and connection info are .env found in the same directory